# **Semiparametric SVM training using subgradients in Spark **

#### bla, bla, bla. 

#### We will benchmark the algorithms with data files from UCI:

* **Ripley**, the Ripley dataset
* **Kwok**, the Kwok dataset
* **Twonorm**, the Twonorm dataset
* **Waveform**, the Waveform dataset
* **Covertype**, the Covertype dataset


In [1]:
# definir variable de usuario y ejecutar condicionalmente cualquier código que dependa del contexto de ejecución

user = 'navia'
#user = 'roberto'

#modelo = 'hybrid' 
#modelo = 'kernelgrad' 

if user == 'roberto':
    # definir sc
    import findspark
    findspark.init()
    from pyspark import SparkConf, SparkContext
    conf = (SparkConf().setMaster("local[4]").setAppName("My app").set("spark.executor.memory", "2g"))
    sc = SparkContext(conf = conf)

if user == 'navia':
    sc.addPyFile("file:///export/usuarios01/navia/spark/SVM_spark/common/lib/svm_utils.py")
    sc.addPyFile("file:///export/usuarios01/navia/spark/SVM_spark/common/lib/quadtree_utils.py")   
    import svm_utils as SVM_UTILS
    import quadtree_utils as QUADTREE
    import numpy as np
    from pyspark.mllib.regression import LabeledPoint
    import pickle
    from math import sqrt
    %matplotlib inline

overwrite_results = False # overwrite results

kdatasets = [1, 2, 3, 4]
folds = [1, 2, 3, 4, 5]

#Niter = 2
Nnodes = 32
Samplefraction = 0.05

Niter = 150
NCs = [5, 10, 25, 50, 100, 200]

#Niter = 300
#NCs = [50, 100, 200]


kdatasets = [1, 2, 3, 4]
folds = [1, 2, 3, 4, 5]
Niters = [50, 100, 200]
NCs = [5, 10, 25, 50, 100, 200]
modelos = ['hybrid', 'kernelgrad']
modelos = ['kernelgrad']
#modelos = ['hybrid']

for modelo in modelos:
    for kdataset in kdatasets:
        for kfold in folds:
            for Niter in Niters:
                x_tr, y_tr, x_val, y_val, x_tst, y_tst, name_dataset = SVM_UTILS.load_data(kdataset, kfold)
                NI = x_tr.shape[1]
                sigma = sqrt(NI)
                C = 10.0
                XtrRDD = sc.parallelize(np.hstack((y_tr, x_tr)), Nnodes).map(lambda x: LabeledPoint(x[0], x[1:len(x)]))
                XvalRDD = sc.parallelize(np.hstack((y_val, x_val)), Nnodes).map(lambda x: LabeledPoint(x[0], x[1:len(x)]))
                XtstRDD = sc.parallelize(np.hstack((y_tst, x_tst)), Nnodes).map(lambda x: LabeledPoint(x[0], x[1:len(x)]))
                for NC in NCs:
                    print "Dataset = %s, modelo = %s, kfold = %d, Niter = %d, NC = %d" % (name_dataset, modelo, kfold, Niter, NC)
                    filename = './results/dataset_' + str(kdataset) + '_modelo_' + modelo + '_NC_' + str(NC) + '_Niter_' + str(Niter) + '_kfold_' + str(kfold) + '.pkl'
                    try:
                        f = open(filename,'r')
                        f.close()
                        file_exists = True
                    except:
                        file_exists = False
                        pass
                    execute = False
                    if file_exists:
                        if overwrite_results:
                            execute = True
                    else:
                        execute = True                                  
                    if execute:
                        
                        if modelo == 'hybrid':
                            auc_val, auc_tst, exe_time = SVM_UTILS.train_hybridSVM(XtrRDD, XvalRDD, XtstRDD, sigma, C, NC, name_dataset, Niter, Samplefraction)
                        
                        if modelo == 'kernelgrad':
                            auc_val, auc_tst, exe_time = SVM_UTILS.train_kernelgrad(XtrRDD, XvalRDD, XtstRDD, sigma, C, NC, name_dataset, Niter, Samplefraction)
                        
                        with open(filename, 'w') as f:
                            pickle.dump([auc_val, auc_tst, exe_time], f)


Dataset = Ripley, modelo = kernelgrad, kfold = 1, Niter = 50, NC = 5
Clustering data...


/opt/spark-1.6.0-bin-2.6.0/python/pyspark/mllib/clustering.py:176: UserWarning: Support for runs is deprecated in 1.6.0. This param will have no effect in 1.7.0.
  "Support for runs is deprecated in 1.6.0. This param will have no effect in 1.7.0.")


Building the kernel expansion...
Training the hybrid SVM model during 50 iterations
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 Fin!
Predicting and evaluating...
AUCval = 0.962963, AUCtst = 0.953204
Elapsed_time = 290.603692
Dataset = Ripley, modelo = kernelgrad, kfold = 1, Niter = 50, NC = 10
Clustering data...
Building the kernel expansion...
Training the hybrid SVM model during 50 iterations
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 Fin!
Predicting and evaluating...
AUCval = 0.961353, AUCtst = 0.959416
Elapsed_time = 257.812434
Dataset = Ripley, modelo = kernelgrad, kfold = 1, Niter = 50, NC = 25
Clustering data...
Building the kernel expansion...
Training the hybrid SVM model during 50 iterations
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 3

KeyboardInterrupt: 